In [2]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_validate, GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_percentage_error
from time import time
from numpy import arange

In [3]:
# Loading the data

df = pd.read_csv('../data/preprocessed/newborntrain_processed.csv', sep=";")
#print(df.head())

# Removing the target variable from the feature

X = df.drop("newborn_weight", axis=1)
Y = df["newborn_weight"]

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=12345)

MLmodel = ElasticNet(alpha=0.5, l1_ratio=0.5)
MLmodel.fit(X_train, Y_train)

ElasticNet(alpha=0.5)

In [6]:
Y_Pred = MLmodel.predict(X_test)

Mape = mean_absolute_percentage_error(Y_test,Y_Pred)
print(Mape)

0.16369967266740842


## Grid Search

In [15]:
# Grid search results to df
def gs_to_df(grid_search):
    df = pd.DataFrame()
    for param in param_grid[0].keys():
        df[param] = 0
    df["mean_train_score"] = df["std_train_score"] = df["mean_test_score"] = df["std_test_score"] = 0
    for i in range(len(grid_search.cv_results_['params'])):
        new_row = grid_search.cv_results_['params'][i] | {"mean_train_score": grid_search.cv_results_['mean_train_score'][i],
                                                            "std_train_score": grid_search.cv_results_['std_train_score'][i],
                                                            "mean_test_score": grid_search.cv_results_['mean_test_score'][i],
                                                            "std_test_score": grid_search.cv_results_['std_test_score'][i]}
        df.loc[len(df)] = new_row

    return df

In [9]:
model = ElasticNet()
# define model evaluation method
cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=12345)
# define grid
grid = dict()
grid['alpha'] = [0.01, 0.1, 1.0, 4.0, 7.0, 10.0]
grid['l1_ratio'] = [0.1, 0.3, 0.5, 0.7, 0.9]
# define search
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_percentage_error', cv=rkf, return_train_score=True)
# perform the search
results = search.fit(X, Y)
# summarize
print('NMAPE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

C:\Users\mraer\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.359e+08, tolerance: 6.687e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\mraer\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.127e+08, tolerance: 6.689e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\mraer\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.224e+08, toleranc

NMAPE: -0.163
Config: {'alpha': 0.01, 'l1_ratio': 0.9}


In [21]:
mean_train_score = results.cv_results_['mean_train_score']
mean_test_score = results.cv_results_['mean_test_score']
std_train_score = results.cv_results_['std_train_score']
std_test_score = results.cv_results_['std_test_score']


# Create a DataFrame to hold the scores
df_scores = pd.DataFrame({
    'Mean Train Score': mean_train_score,
    'Mean Test Score': mean_test_score,
    'Std Train Score': std_train_score,
    'Std Test Score': std_test_score
})

sorted_df = df_scores.sort_values(by="Mean Test Score", ascending=False)

sorted_df.to_csv('./Validation/Elastic_Net_metrics_2_5_cv.csv', index=False,sep=';')

# Print the DataFrame
print(sorted_df)



    Mean Train Score  Mean Test Score  Std Train Score  Std Test Score
4          -0.163120        -0.163122         0.000148        0.000599
3          -0.163121        -0.163123         0.000148        0.000599
2          -0.163122        -0.163124         0.000148        0.000599
1          -0.163124        -0.163125         0.000148        0.000599
0          -0.163125        -0.163126         0.000148        0.000599
9          -0.163126        -0.163128         0.000148        0.000600
8          -0.163144        -0.163146         0.000148        0.000601
7          -0.163168        -0.163169         0.000148        0.000602
6          -0.163194        -0.163195         0.000149        0.000603
5          -0.163222        -0.163223         0.000149        0.000604
14         -0.163263        -0.163264         0.000149        0.000605
13         -0.163534        -0.163535         0.000150        0.000611
12         -0.163734        -0.163735         0.000151        0.000614
19    